<a href="https://colab.research.google.com/github/LorenzoZInna/project_doppelganger/blob/Data-Collection-Preparation-from-GCP---Lorenzo/notebooks/data_for_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

## Read Data from GCP

In [ ]:
import pandas as pd
#this is a public file, it requires no authentication
file_path = 'https://storage.googleapis.com/doppelganger-1-bucket/Kaggle%20Spotify%20Database/tracks.csv'
spotify_data = pd.read_csv(file_path)
spotify_data

In [ ]:
print(spotify_data.info())

In [ ]:
missing_name = spotify_data.isnull()

In [ ]:
missing_name

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
586668,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
586669,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
586670,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
spotify_data_cleaned = spotify_data.dropna(subset=['name']).reset_index(drop=True)

In [ ]:
print(spotify_data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586601 entries, 0 to 586600
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586601 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586601 non-null  int64  
 3   duration_ms       586601 non-null  int64  
 4   explicit          586601 non-null  int64  
 5   artists           586601 non-null  object 
 6   id_artists        586601 non-null  object 
 7   release_date      586601 non-null  object 
 8   danceability      586601 non-null  float64
 9   energy            586601 non-null  float64
 10  key               586601 non-null  int64  
 11  loudness          586601 non-null  float64
 12  mode              586601 non-null  int64  
 13  speechiness       586601 non-null  float64
 14  acousticness      586601 non-null  float64
 15  instrumentalness  586601 non-null  float64
 16  liveness          58

In [ ]:
print(spotify_data_cleaned['explicit'].value_counts())

0    560737
1     25864
Name: explicit, dtype: int64


In [ ]:
print(spotify_data_cleaned['mode'].value_counts())

1    386451
0    200150
Name: mode, dtype: int64


In [ ]:
print(spotify_data_cleaned['time_signature'].value_counts())

4    503770
3     64498
5     11394
1      6602
0       337
Name: time_signature, dtype: int64


In [ ]:
print(spotify_data_cleaned['valence'].value_counts())

0.96100    2679
0.96200    2312
0.96300    2023
0.96400    1846
0.96000    1651
           ... 
0.00704       1
0.00831       1
0.00382       1
0.00789       1
0.02030       1
Name: valence, Length: 1805, dtype: int64


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape the 'popularity' column to a 2D array, as required by the scaler
popularity_values = spotify_data_cleaned['popularity'].values.reshape(-1, 1)

# Fit and transform the 'popularity' column using the scaler
spotify_data_cleaned['popularity_normalized'] = scaler.fit_transform(popularity_values)

# Drop the original 'popularity' column if you no longer need it
# spotify_data_cleaned.drop('popularity', axis=1, inplace=True)

# Display the updated DataFrame
print(spotify_data_cleaned[['popularity', 'popularity_normalized']].head())

   popularity  popularity_normalized
0           6                   0.06
1           0                   0.00
2           0                   0.00
3           0                   0.00
4           0                   0.00


In [ ]:
spotify_data_cleaned = pd.get_dummies(spotify_data_cleaned, columns=['key'], prefix='key')

In [ ]:
spotify_data_cleaned = pd.get_dummies(spotify_data_cleaned, columns=['mode'], prefix='mode')

In [ ]:
spotify_data_cleaned = pd.get_dummies(spotify_data_cleaned, columns=['time_signature'], prefix='time_signature')

In [ ]:
spotify_data_cleaned.set_index('id', inplace=True)

In [ ]:
# Remove square brackets and single quotes from the 'artists' column and lowercase the name column too but I think is a good idea to leave the names like that
spotify_data_cleaned['artists'] = spotify_data_cleaned['artists'].str.strip("[]").str.replace("'", "").str.lower()
spotify_data_cleaned['name'] = spotify_data_cleaned['name'].str.lower()

In [ ]:
# Convert 'release_date' to datetime format
spotify_data_cleaned['release_date'] = pd.to_datetime(spotify_data_cleaned['release_date'], errors='coerce')

# Extract additional features: year, month, day
spotify_data_cleaned['release_year'] = spotify_data_cleaned['release_date'].dt.year
spotify_data_cleaned['release_month'] = spotify_data_cleaned['release_date'].dt.month
spotify_data_cleaned['release_day'] = spotify_data_cleaned['release_date'].dt.day

# Display the updated DataFrame
spotify_data_cleaned

,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,loudness,...,mode_0,mode_1,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,release_year,release_month,release_day
id,,,,,,,,,,,,,,,,,,,,,
35iwgR4jXetI318WEWsa1Q,carve,6,126903,0,uli,['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,-13.338,...,0,1,0,0,1,0,0,1922,2,22
021ht4sdgPcrDgSk7JTbKY,capítulo 2.16 - banquero anarquista,0,98200,0,fernando pessoa,['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,-22.136,...,0,1,0,1,0,0,0,1922,6,1
07A5yehtSnoedViJAZkNnc,vivo para quererte - remasterizado,0,181640,0,ignacio corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,-21.180,...,0,1,0,0,0,0,1,1922,3,21
08FmqUhxtyLTn6pAh6bk45,el prisionero - remasterizado,0,176907,0,ignacio corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,-27.961,...,0,1,0,0,1,0,0,1922,3,21
08y9GfoqCWfOGsKdwojr5e,lady of the evening,0,163080,0,dick haymes,['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.402,0.1580,-16.900,...,1,0,0,0,0,1,0,1922,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,阿yueyue,['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,-7.471,...,1,0,0,0,0,1,0,2020,9,26
0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,role model,['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,-5.223,...,0,1,0,0,0,1,0,2020,10,21
27Y1N4Q4U3EfDU5Ubw8ws2,what they'll say about us,70,187601,0,finneas,['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,-12.823,...,1,0,0,0,0,1,0,2020,9,2


In [ ]:
spotify_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586601 entries, 35iwgR4jXetI318WEWsa1Q to 5Ocn6dZ3BJFPWh4ylwFXtn
Data columns (total 39 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   name                   586601 non-null  object        
 1   popularity             586601 non-null  int64         
 2   duration_ms            586601 non-null  int64         
 3   explicit               586601 non-null  int64         
 4   artists                586601 non-null  object        
 5   id_artists             586601 non-null  object        
 6   release_date           586601 non-null  datetime64[ns]
 7   danceability           586601 non-null  float64       
 8   energy                 586601 non-null  float64       
 9   loudness               586601 non-null  float64       
 10  speechiness            586601 non-null  float64       
 11  acousticness           586601 non-null  float64       
 12  instrumental

In [ ]:
# Using ['valence'] to label the songs
valence_ranges = {
    (0, 0.2): 'Extremely Sad',
    (0.2, 0.4): 'Very Sad',
    (0.4, 0.6): 'Sad',
    (0.6, 0.8): 'A Bit Happy',
    (0.8, 1.0): 'Very Happy'
}

# Create a new column 'emotion' based on 'valence' ranges
spotify_data_cleaned['emotion'] = pd.cut(spotify_data_cleaned['valence'], bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0], labels=valence_ranges.values(), include_lowest=True)

# Display the distribution of emotion labels
print(spotify_data_cleaned['emotion'].value_counts())

A Bit Happy      142629
Sad              135560
Very Happy       124774
Very Sad         118550
Extremely Sad     65088
Name: emotion, dtype: int64


In [ ]:
spotify_data_cleaned

,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,loudness,...,mode_1,time_signature_0,time_signature_1,time_signature_3,time_signature_4,time_signature_5,release_year,release_month,release_day,emotion
id,,,,,,,,,,,,,,,,,,,,,
35iwgR4jXetI318WEWsa1Q,carve,6,126903,0,uli,['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,-13.338,...,1,0,0,1,0,0,1922,2,22,Extremely Sad
021ht4sdgPcrDgSk7JTbKY,capítulo 2.16 - banquero anarquista,0,98200,0,fernando pessoa,['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,-22.136,...,1,0,1,0,0,0,1922,6,1,A Bit Happy
07A5yehtSnoedViJAZkNnc,vivo para quererte - remasterizado,0,181640,0,ignacio corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,-21.180,...,1,0,0,0,0,1,1922,3,21,Sad
08FmqUhxtyLTn6pAh6bk45,el prisionero - remasterizado,0,176907,0,ignacio corsini,['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,-27.961,...,1,0,0,1,0,0,1922,3,21,Very Sad
08y9GfoqCWfOGsKdwojr5e,lady of the evening,0,163080,0,dick haymes,['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.402,0.1580,-16.900,...,0,0,0,0,1,0,1922,1,1,Extremely Sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,阿yueyue,['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,-7.471,...,0,0,0,0,1,0,2020,9,26,Very Sad
0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,role model,['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,-5.223,...,1,0,0,0,1,0,2020,10,21,A Bit Happy
27Y1N4Q4U3EfDU5Ubw8ws2,what they'll say about us,70,187601,0,finneas,['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,-12.823,...,0,0,0,0,1,0,2020,9,2,Extremely Sad
